In [1]:
import pandas as pd

In [2]:
#Load Data
xls = pd.ExcelFile('Adops & Data Scientist Sample Data.xlsx')
df1 = pd.read_excel(xls, 'Q1 Analytics')
#df2 = pd.read_excel(xls, 'Q2 Regression', header=None)

### Question 1.1

In [3]:
df1_BDV = df1[df1['country_id']=='BDV'] # filter by country id equal BDV

In [4]:
df1_BDV.shape # As said in question we filtered 844 rows

(844, 4)

####  site_id with the largest number of unique users and the number of unique users

In [5]:
pd.DataFrame(df1_BDV.groupby('site_id')['user_id'].nunique()).sort_values('user_id').tail(1)

,user_id
site_id,
5NPAU,544


### Question 1.2 

In [6]:
df1.dtypes #we see that the datatype of timestamp is object

ts            object
user_id       object
country_id    object
site_id       object
dtype: object

In [7]:
df1['ts'] = pd.to_datetime(df1['ts']) #converting to datetime format

In [8]:
df_q2 = df1[(df1['ts'] > '2019-02-03 00:00:00') & (df1['ts'] < '2019-02-04 23:59:59')] #filter wrt the timestamp

In [9]:
df_count = df_q2.groupby(['user_id','site_id'])['ts'].count().to_frame().reset_index()

In [10]:
df_count[df_count.ts > 10] # filter by visit times greater than 10

,user_id,site_id,ts
3,LC06C3,N0OTG,25
417,LC3A59,N0OTG,26
485,LC3C7E,3POLC,15
493,LC3C9D,N0OTG,17


### Question 1.3 

In [11]:
# for each user, get last visit time
df_last_visit = df1.groupby('user_id')['ts'].max().to_frame().reset_index()

In [12]:
# left join previous table with the full table to get last visit site
df_last_site = df_last_visit.merge(df1, on = ['ts','user_id'], how='left')

In [13]:
# change column names
df_last_site.columns = ['user_id','num_of_users', 'country_last', 'site_id']

In [15]:
# group by last vist site and count each sites' occurrence, sort by counts
df_last_site.groupby('site_id')['num_of_users'].count().to_frame().sort_values('num_of_users', ascending = False).reset_index().head(3)

,site_id,num_of_users
0,5NPAU,992
1,N0OTG,561
2,QGO3G,289


### Question 1.4 

In [17]:
# for each user, get first visit time
df_first_visit = df1.groupby('user_id')['ts'].min().to_frame().reset_index()

In [19]:
# left join previous table with the full table to get first visit site
df_first_site = df_first_visit.merge(df1, on = ['ts','user_id'], how='left' )

In [20]:
# change column names
df_first_site.columns = ['user_id','ts_first', 'country_first', 'site_first']

In [21]:
# combine with the last visit table we get from question 1.3, join by using user_id
df_comb = df_last_site.merge(df_first_site, on ='user_id', how = 'left')

In [25]:
# filter by last visit site equal first visit site and compute the length
len(df_comb[df_comb.site_first == df_comb.site_id])

1670